In [1]:
import numpy as np

In [11]:
# if A = UDV' then
# A'A = UDV' VDU'= U(D^2)U'
# therefore eig(A'A) == svd(A'A)
# because:
# A'A U = U(D^2) = (D^2) U
# which is of the form
# AX = lambda X

class PCA:
    def __init__(self,n_components):
        self.n_components = n_components
        self.mean = None
        self.components = None
        
    def fit(self, X):
        mu = X.mean(axis=0)
        X_ = X - mu
        cov = np.matmul(X_.T, X_) # (d, d)
        # eigvectors, eigvalues = np.linalg.eig(cov) #eigvectors is column-major
        U, D, Vt = np.linalg.svd(cov)
        eigenvectors = U
        eigenvalues = D
        # sort by highest eigvalues
        idx = np.argsort(eigenvalues)[::-1] # descending
        print(idx)
        components = eigenvectors[:, idx].T
        self.components = components[:self.n_components, :] # n_comp x d
        self.mean = mu
        
    def transform(self, X):
        X_ = X - self.mean # n x d
        # project X_ onto components
        coef = np.matmul(X_, self.components.T)